In [1]:
import sys
sys.path.append('..')
import ScraperFC as sfc
import traceback
import pandas as pd
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm
import itertools
from IPython.display import HTML

Missing a GK table
* https://fbref.com/en/matches/0d0210e0/Wimbledon-Tottenham-Hotspur-October-24-1992-Premier-League
* https://fbref.com/en/matches/7b05eda0/Atalanta-Como-December-31-1988-Serie-A

Matchweek el is NoneType?
* https://fbref.com/en/matches/aba86414/Blackburn-Rovers-Newcastle-United-January-17-2009-Premier-League

# Test the match-scraping functions

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import re
from datetime import datetime

scraper = sfc.FBRef()

try:
    leagues = ['EPL', 'La Liga', 'Bundesliga', 'Serie A', 'Ligue 1', 'MLS']
    years = range(1988, 2024)
    iterator = list(itertools.product(leagues, years))
    iterator.remove(('MLS',2023))
    random_idxs = np.random.choice(len(iterator), size=len(iterator)//20, replace=False)
    random_seasons = np.array(iterator)[random_idxs]

    for (league, year) in tqdm(iterator[54:]):
        year = int(year) # year became a string during random sampling?
        league = str(league) # league also became a weird type of numpy string?
        print(year, league)

        err, valid = sfc.shared_functions.check_season(year,league,'FBRef')
        if not valid:
            print(err)
            continue

        links = scraper.get_match_links(year,league)
        matches = scraper.scrape_matches(year, league)
#         ########################################################################
#         ########################################################################
#         matches = pd.DataFrame()
#         for link in tqdm(links, position=1, leave=False):

#             got_link = False # Don't proceed until we've successfully retrieved the page
#             while not got_link:
#                 response = requests.get(link)
#                 time.sleep(5)
#                 if response.status_code not in [403, 500]:
#                     # 200 - OK
#                     # 403 - file not found
#                     # 500 - server error
#                     got_link = True
#             soup = BeautifulSoup(response.content, 'html.parser')

#             """ Matchweek """
#             matchweek_el = list(soup.find('a', {'href': re.compile('-Stats')}, text=True).parents)[0]
#             if '-Major-League-Soccer' in link:
#                 matchweek = matchweek_el\
#                     .getText()\
#                     .replace('Major League Soccer','')\
#                     .replace(')','')\
#                     .replace('(', '')\
#                     .strip()
#             else:
#                 matchweek = int(
#                     matchweek_el\
#                     .getText()\
#                     .split('(')[-1]\
#                     .split(')')[0]\
#                     .lower()\
#                     .replace('matchweek','')\
#                     .strip()
#                 )

#             """ Team names and ids """
#             team_els = [el.find('a') \
#                     for el in soup.find('div', {'class': 'scorebox'}).find_all('strong') \
#                     if el.find('a', href=True) is not None][:2]
#             home_team_name = team_els[0].getText()
#             home_team_id   = team_els[0]['href'].split('/')[3]
#             away_team_name = team_els[1].getText()
#             away_team_id   = team_els[1]['href'].split('/')[3]

#             """ Formations """
#             lineups = [pd.read_html(str(el.find('table')))[0] \
#                        for el in soup.find_all('div', {'class': 'lineup'})]


#             """ Player stats """
#             player_stats = dict()
#             for i, (team, team_id) in enumerate([('Home',home_team_id), ('Away',away_team_id)]):

#                 summary = soup.find_all('table', {'id': re.compile(f'stats_{team_id}_summary')})
#                 assert len(summary) < 2

#                 gk = soup.find_all('table', {'id': re.compile(f'keeper_stats_{team_id}')})
#                 assert len(gk) < 2

#                 passing = soup.find_all('table', {'id': re.compile(f'stats_{team_id}_passing$')})
#                 assert len(passing) < 2

#                 pass_types = soup.find_all('table', {'id': re.compile(f'stats_{team_id}_passing_types')})
#                 assert len(pass_types) < 2

#                 defense = soup.find_all('table', {'id': re.compile(f'stats_{team_id}_defense')})
#                 assert len(defense) < 2

#                 possession = soup.find_all('table', {'id': re.compile(f'stats_{team_id}_possession')})
#                 assert len(possession) < 2

#                 misc = soup.find_all('table', {'id': re.compile(f'stats_{team_id}_misc')})
#                 assert len(misc) < 2

#                 player_stats[team] = {
#                     'Team Sheet': lineups[i] if len(lineups)!=0 else None,
#                     'Summary': pd.read_html(str(summary[0]))[0] if len(summary)==1 else None,
#                     'GK': pd.read_html(str(gk[0]))[0] if len(gk)==1 else None,
#                     'Passing': pd.read_html(str(passing[0]))[0] if len(passing)==1 else None,
#                     'Pass Types': pd.read_html(str(pass_types[0]))[0] if len(pass_types)==1 else None,
#                     'Defense': pd.read_html(str(defense[0]))[0] if len(defense)==1 else None,
#                     'Possession': pd.read_html(str(possession[0]))[0] if len(possession)==1 else None,
#                     'Misc': pd.read_html(str(misc[0]))[0] if len(misc)==1 else None,
#                 }

#             """ Shots """
#             both_shots = soup.find_all('table', {'id': 'shots_all'})
#             if len(both_shots) == 1:
#                 both_shots = pd.read_html(str(both_shots[0]))[0]
#                 both_shots = both_shots[~both_shots.isna().all(axis=1)]
#             else:
#                 both_shots = None
#             home_shots = soup.find_all('table', {'id': f'shots_{home_team_id}'})
#             if len(home_shots) == 1:
#                 home_shots = pd.read_html(str(home_shots[0]))[0]
#                 home_shots = home_shots[~home_shots.isna().all(axis=1)]
#             else:
#                 home_shots = None
#             away_shots = soup.find_all('table', {'id': f'shots_{away_team_id}'})
#             if len(away_shots) == 1:
#                 away_shots = pd.read_html(str(away_shots[0]))[0]
#                 away_shots = away_shots[~away_shots.isna().all(axis=1)]
#             else:
#                 away_shots = None

#             match = pd.Series(dtype=object)
#             match['Link'] = link
#             match['Date'] = datetime.strptime(
#                 str(soup.find('h1')).split('<br/>')[0].split('–')[-1].replace('</h1>','').strip(),
#                 '%A %B %d, %Y'
#             ).date()
#             match['Matchweek'] = matchweek
#             match['Home Team'] = home_team_name
#             match['Away Team'] = away_team_name
#             match['Home Team ID'] = home_team_id
#             match['Away Team ID'] = away_team_id
#             if len(lineups) > 0:
#                 match['Home Formation'] = lineups[0].columns[0].split('(')[-1].replace(')','').strip()
#                 match['Away Formation'] = lineups[1].columns[0].split('(')[-1].replace(')','').strip()
#             match['Home Goals'] = player_stats['Home']['Summary'][('Performance','Gls')].values[-1]
#             match['Away Goals'] = player_stats['Away']['Summary'][('Performance','Gls')].values[-1]
#             match['Home Ast'] = player_stats['Home']['Summary'][('Performance','Ast')].values[-1]
#             match['Away Ast'] = player_stats['Away']['Summary'][('Performance','Ast')].values[-1]
#             if 'Expected' in player_stats['Home']['Summary'].columns.get_level_values(0):
#                 match['Home xG']   = player_stats['Home']['Summary'][('Expected','xG')].values[-1]
#                 match['Away xG']   = player_stats['Away']['Summary'][('Expected','xG')].values[-1]
#                 match['Home npxG'] = player_stats['Home']['Summary'][('Expected','npxG')].values[-1]
#                 match['Away npxG'] = player_stats['Away']['Summary'][('Expected','npxG')].values[-1]
#                 match['Home xA']   = player_stats['Home']['Summary'][('Expected','xA')].values[-1]
#                 match['Away xA']   = player_stats['Away']['Summary'][('Expected','xA')].values[-1]
#             match['Home Player Stats'] = pd.Series(player_stats['Home'])
#             match['Away Player Stats'] = pd.Series(player_stats['Away'])
#             match['Shots'] = pd.Series({
#                 'Both': both_shots,
#                 'Home': home_shots,
#                 'Away': away_shots,
#             })
#             match = match.to_frame().T
            
#             matches = pd.concat([matches, match], ignore_index=True)
#         ########################################################################
#         ########################################################################
        assert matches.shape[0] == len(links)

        clear_output()
    print('DONE')
except:
    traceback.print_exc()
        
scraper.close()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST driver version for 104.0.5112
[WDM] - Driver [C:\Users\Owner\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache
  0%|                                                                                          | 0/161 [00:00<?, ?it/s]

2006 La Liga
Gathering match links.



 27%|█████████████████████▍                                                         | 517/1900 [00:07<00:20, 68.48it/s]


 51%|███████████████████████████████████████▉                                       | 960/1900 [00:14<00:17, 52.77it/s]


 69%|██████████████████████████████████████████████████████▏                       | 1319/1900 [00:22<00:12, 45.25it/s]


...

Scraping matches.



 17%|█████████████▊                                                                   | 65/380 [06:49<33:21,  6.36s/it]


 34%|███████████████████████████▌                                                    | 131/380 [13:39<27:55,  6.73s/it]


 52%|█████████████████████████████████████████▍                                      | 197/380 [20:30<18:51,  6.19s/it]


 63%|██████████████████████████████████████████████████                              | 238/380 [24:45<14:25,  6.10s/it]

In [ ]:
iterator[54:]

In [ ]:
# print(link)
# print(response.status_code)
# HTML(response.text)